In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt

In [25]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
s1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
test_data = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='test.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [3]:
data_list = [s1, s2, s3, s4, s5]

In [4]:
#s1-s5でranking scoreとlabelをlistに分割する
ranking=[]
label=[]
for doc in range(len(data_list)):
    ranking.append([])
    label.append([])
    for query in range(len(data_list[doc])):
        _, batch_ranking, std_label = data_list[doc][query]
        ranking[-1].append(batch_ranking)
        label[-1].append(std_label)

In [5]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [42]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
                
        # computes p_ij
        batch_pij = 1 / (1 + torch.exp(-batch_sij)) # [40, 40]
                
        # computes loss
        batch_loss = 0.5 * (1-batch_Sij) * batch_sij + torch.log(1+torch.exp(-batch_sij)) #[40, 40]
                
        # make upper triangular matrix
        batch_loss_triu = torch.triu(batch_loss, diagonal=1)

        # computes the mean value of a
        num_elements = (batch_loss_triu.size(1)*batch_loss_triu.size(1)-batch_loss_triu.size(1))/2
        batch_loss_mean = torch.sum(batch_loss)/num_elements
        
        #loss_list.append(batch_loss_mean)
                
        #print("batch_loss_mean", batch_loss_mean)
        
        return batch_loss_mean
    
    def predict(self, x):
        return self.model(x)

In [43]:
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

In [44]:
def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

In [45]:
def compute_ndcg(rank,label,cutoff):     
    t = model.predict(rank)
    r = torch.argsort(t,dim=1,descending = True)
    ideal,_ = torch.sort(label,1,descending = True)
    ideal = torch.unsqueeze(ideal,2).reshape(-1)
    pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
    ndcg = torch.nan_to_num(nDCG(ideal,pred,cutoff))
    return ndcg

In [46]:
def training_loop(n_epochs,optimizer,model,ranking_score,std_label):
    count = 0
    best_ndcg = 0
    
    while count < 5:
        
        print("fold:{0}".format(count+1))
        # 最初の3つをtrainに
        train = 2
        # 1つをvalidationに
        val = 3
        # 最後をtestに
        test = 4
    
        # モデルの訓練
        print("*train*")
        for epoch in range(1, n_epochs + 1):
        
            # クエリごとの訓練
            batch_loss = 0
            batch_ndcg = 0
            
            test_ranking = ranking[count]+ranking[(count+1)%5]+ranking[(count+2)%5]
            test_label = label[count]+label[(count+1)%5]+label[(count+2)%5]
            
            for test_query in range(len(test_ranking)):
                batch_loss += model.forward(batch_ranking=test_ranking[test_query], label=test_label[test_query])
                batch_ndcg += compute_ndcg(test_ranking[test_query],test_label[test_query],5)
                    
            fold_loss_mean = batch_loss / len(test_ranking)
            fold_ndcg_mean = batch_ndcg / len(test_ranking)
            
            if epoch%10==0:
                print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, fold_loss_mean, fold_ndcg_mean))
            
            # パラメータ選択
            val_loss = 0
            val_ndcg = 0
            for val_query in range(len(ranking[(count+val)%5])):
                val_loss += model.forward(batch_ranking=ranking[(count+val)%5][val_query], label=label[(count+val)%5][val_query])
                val_ndcg += compute_ndcg(ranking[(count+val)%5][val_query],label[(count+val)%5][val_query],5)
                
            val_loss_mean = val_loss / len(ranking[(count+val)%5])
            val_ndcg_mean = val_ndcg / len(ranking[(count+val)%5])
            
            
            if val_ndcg_mean > best_ndcg:
                best_ndcg = val_ndcg_mean
                
                #print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, val_loss_mean, val_ndcg_mean))
            
                # 訓練したモデルの保存
                torch.save(model.state_dict(), 'weight.pth')
            
            optimizer.zero_grad()
            fold_loss_mean.backward()
            optimizer.step()
            
        # パラメータの読み込み
        param = torch.load('weight.pth')
        model.load_state_dict(param)
        
        
        # test(関数を使わないバージョンだよーーーー)
        print("*test*")    
        with torch.no_grad():
            test_ndcg = 0
            test_loss = 0
            for test_query in range(len(ranking[(count+test)%5])):
            #computes ndcg
                test_loss += model.forward(batch_ranking=ranking[(count+test)%5][test_query], label=label[(count+test)%5][test_query])
                test_ndcg += compute_ndcg(ranking[(count+test)%5][test_query],label[(count+test)%5][test_query],5)
                  
            test_ndcg_mean = test_ndcg / len((ranking[(count+test)%5]))
        
        print('loss:{0}, ndcg:{1}'.format(val_loss_mean, val_ndcg_mean))
        
        count += 1
        print("*****************************")

In [47]:
# クラスのインスタンス化
model = RankNet(layers)

In [48]:
training_loop(
    n_epochs = 50,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    ranking_score=ranking,
    std_label=label,
    )

fold:1
*train*
epoch:10, loss:1.4197638034820557, ndcg:0.33790695667266846
epoch:20, loss:1.4197347164154053, ndcg:0.33813315629959106
epoch:30, loss:1.4197053909301758, ndcg:0.3382973074913025
epoch:40, loss:1.4196765422821045, ndcg:0.33977100253105164
epoch:50, loss:1.4196473360061646, ndcg:0.3405761420726776
*test*
loss:1.4200077056884766, ndcg:0.3776305913925171
*****************************
fold:2
*train*
epoch:10, loss:1.419848918914795, ndcg:0.35257819294929504
epoch:20, loss:1.419819951057434, ndcg:0.3530759811401367
epoch:30, loss:1.4197890758514404, ndcg:0.35422617197036743
epoch:40, loss:1.4197595119476318, ndcg:0.3548951745033264
epoch:50, loss:1.4197300672531128, ndcg:0.3554001450538635
*test*
loss:1.4195078611373901, ndcg:0.37217429280281067
*****************************
fold:3
*train*
epoch:10, loss:1.4199048280715942, ndcg:0.359062135219574
epoch:20, loss:1.4198771715164185, ndcg:0.36027106642723083
epoch:30, loss:1.419848918914795, ndcg:0.36040839552879333
epoch:40, lo